# Bases de Datos II. NoSQL (MongoDB sesión 3)

```
                                                                  ,gggggggggggg,    ,ggggggggggg,  
                                                                 dP"""88""""""Y8b, dP"""88""""""Y8,
                                                                 Yb,  88       `8b,Yb,  88      `8b
                                                                  `"  88        `8b `"  88      ,8P
                                                                      88         Y8     88aaaad8P"
  ,ggg,,ggg,,ggg,     ,ggggg,    ,ggg,,ggg,     ,gggg,gg    ,ggggg,   88         d8     88""""Y8ba
 ,8" "8P" "8P" "8,   dP"  "Y8ggg,8" "8P" "8,   dP"  "Y8I   dP"  "Y8ggg88        ,8P     88      `8b
 I8   8I   8I   8I  i8'    ,8I  I8   8I   8I  i8'    ,8I  i8'    ,8I  88       ,8P'     88      ,8P
,dP   8I   8I   Yb,,d8,   ,d8' ,dP   8I   Yb,,d8,   ,d8I ,d8,   ,d8'  88______,dP'      88_____,d8'
8P'   8I   8I   `Y8P"Y8888P"   8P'   8I   `Y8P"Y8888P"888P"Y8888P"   888888888P"       88888888P"  
                                                    ,d8I'                                          
                                                  ,dP'8I                                           
                                                 ,8"  8I                                           
                                                 I8   8I                                           
                                                 `8, ,8I                                           
                                                  `Y8P"                             
                                                  
                                                                 
                                             ______
|_  _  _  _  _     _| _     _| _ _|_ _  _     |  |
|_)(_|_> (/__>    (_|(/_   (_|(_| |_(_)_>    _|__|_
```


![MongoDB](https://webassets.mongodb.com/_com_assets/cms/MongoDB_Logo_FullColorBlack_RGB-4td3yuxzjs.png)

Esta hoja es la tercera entrega y última del trabajo con MongoDB. Veremos temas avanzados como cálculos `aggregate()` más complejos, el uso de índices, etc. Al final habrá un conjunto de ejercicios que habrá que resolver y entregar en la tarea abierta para la ocasión.

## Inicio de la hoja


In [ ]:
RunningInCOLAB = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False

In [ ]:
!wget -qO - https://www.mongodb.org/static/pgp/server-6.0.asc | sudo gpg --dearmor > /etc/apt/trusted.gpg.d/mongo-server-6.gpg

In [ ]:
%%bash
sudo adduser --system --no-create-home mongodb
sudo addgroup --system mongodb
sudo adduser mongodb mongodb

# create db -- note: this should agree with dbpath in mongod.conf
if [ ! -d /var/lib/mongodb ]; then
  sudo mkdir -p /var/lib/mongodb
  sudo chown mongodb:mongodb /var/lib/mongodb
fi

# create logdir -- note: this should agree with logpath in mongod.conf
if [ ! -d /var/log/mongodb ]; then
  sudo mkdir -p /var/log/mongodb
  sudo chown mongodb:mongodb /var/log/mongodb
fi

In [ ]:
!echo "deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-6.0.list

In [ ]:
!sudo apt-get update -qq

In [ ]:
!sudo ln -sf /bin/true /bin/systemctl

In [ ]:
!sudo apt-get install -y -qq dialog mongodb-org

In [ ]:
!sudo /usr/bin/mongod --config /etc/mongod.conf --fork

In [ ]:
!sudo mongod --version

In [ ]:
!pip install --upgrade pymongo

In [ ]:
from pprint import pprint as pp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
matplotlib.style.use('ggplot')

Usaremos la librería `pymongo` para python. La cargamos a continuación.

In [ ]:
import pymongo
from pymongo import MongoClient

La conexión se inicia con `MongoClient` en el `host` descrito en el fichero `docker-compose.yml` (`mongo`), o bien a `localhost` si lo estamos haciendo en Colab.

In [ ]:
db_hostname = "localhost" if RunningInCOLAB else "mongo"

In [ ]:
client = MongoClient(db_hostname,27017)
client

In [ ]:
client.list_database_names()

 - Format: 7zipped
 - Files:
   - **badges**.xml
       - UserId, e.g.: "420"
       - Name, e.g.: "Teacher"
       - Date, e.g.: "2008-09-15T08:55:03.923"
   - **comments**.xml
       - Id
       - PostId
       - Score
       - Text, e.g.: "@Stu Thompson: Seems possible to me - why not try it?"
       - CreationDate, e.g.:"2008-09-06T08:07:10.730"
       - UserId
   - **posts**.xml
       - Id
       - PostTypeId
          - 1: Question
          - 2: Answer
       - ParentID (only present if PostTypeId is 2)
       - AcceptedAnswerId (only present if PostTypeId is 1)
       - CreationDate
       - Score
       - ViewCount
       - Body
       - OwnerUserId
       - LastEditorUserId
       - LastEditorDisplayName="Jeff Atwood"
       - LastEditDate="2009-03-05T22:28:34.823"
       - LastActivityDate="2009-03-11T12:51:01.480"
       - CommunityOwnedDate="2009-03-11T12:51:01.480"
       - ClosedDate="2009-03-11T12:51:01.480"
       - Title=
       - Tags=
       - AnswerCount
       - CommentCount
       - FavoriteCount
   - **posthistory**.xml
	   - Id
	   - PostHistoryTypeId
			- 1: Initial Title - The first title a question is asked with.
			- 2: Initial Body - The first raw body text a post is submitted with.
			- 3: Initial Tags - The first tags a question is asked with.
			- 4: Edit Title - A question's title has been changed.
			- 5: Edit Body - A post's body has been changed, the raw text is stored here as markdown.
			- 6: Edit Tags - A question's tags have been changed.
			- 7: Rollback Title - A question's title has reverted to a previous version.
			- 8: Rollback Body - A post's body has reverted to a previous version - the raw text is stored here.
			- 9: Rollback Tags - A question's tags have reverted to a previous version.
			- 10: Post Closed - A post was voted to be closed.
			- 11: Post Reopened - A post was voted to be reopened.
			- 12: Post Deleted - A post was voted to be removed.
			- 13: Post Undeleted - A post was voted to be restored.
			- 14: Post Locked - A post was locked by a moderator.
			- 15: Post Unlocked - A post was unlocked by a moderator.
			- 16: Community Owned - A post has become community owned.
			- 17: Post Migrated - A post was migrated.
			- 18: Question Merged - A question has had another, deleted question merged into itself.
			- 19: Question Protected - A question was protected by a moderator
			- 20: Question Unprotected - A question was unprotected by a moderator
			- 21: Post Disassociated - An admin removes the OwnerUserId from a post.
			- 22: Question Unmerged - A previously merged question has had its answers and votes restored.
		- PostId
		- RevisionGUID: At times more than one type of history record can be recorded by a single action.  All of these will be grouped using the same RevisionGUID
		- CreationDate: "2009-03-05T22:28:34.823"
		- UserId
		- UserDisplayName: populated if a user has been removed and no longer referenced by user Id
		- Comment: This field will contain the comment made by the user who edited a post
		- Text: A raw version of the new value for a given revision
			- If PostHistoryTypeId = 10, 11, 12, 13, 14, or 15  this column will contain a JSON encoded string with all users who have voted for the PostHistoryTypeId
			- If PostHistoryTypeId = 17 this column will contain migration details of either "from <url>" or "to <url>"
		- CloseReasonId
			- 1: Exact Duplicate - This question covers exactly the same ground as earlier questions on this topic; its answers may be merged with another identical question.
			- 2: off-topic
			- 3: subjective
			- 4: not a real question
			- 7: too localized
   - **postlinks**.xml
     - Id
     - CreationDate
     - PostId
     - RelatedPostId
     - PostLinkTypeId
       - 1: Linked
       - 3: Duplicate
   - **users**.xml
     - Id
     - Reputation
     - CreationDate
     - DisplayName
     - EmailHash
     - LastAccessDate
     - WebsiteUrl
     - Location
     - Age
     - AboutMe
     - Views
     - UpVotes
     - DownVotes
   - **votes**.xml
     - Id
     - PostId
     - VoteTypeId
        - ` 1`: AcceptedByOriginator
        - ` 2`: UpMod
        - ` 3`: DownMod
        - ` 4`: Offensive
        - ` 5`: Favorite - if VoteTypeId = 5 UserId will be populated
        - ` 6`: Close
        - ` 7`: Reopen
        - ` 8`: BountyStart
        - ` 9`: BountyClose
        - `10`: Deletion
        - `11`: Undeletion
        - `12`: Spam
        - `13`: InformModerator
     - CreationDate
     - UserId (only for VoteTypeId 5)
     - BountyAmount (only for VoteTypeId 9)

Descargamos los ficheros CSV del repositorio de datos y los descomprimimos.

In [ ]:
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.csv.7z.001
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.csv.7z.002

In [ ]:
!7zr x es.stackoverflow.csv.7z.001

In [ ]:
!head Users.csv

Las bases de datos se crean conforme se nombran. Se puede utilizar la notación punto o la de diccionario. Las colecciones también.

In [ ]:
db = client.stackoverflow
db = client['stackoverflow']
db

Las bases de datos están compuestas por un conjunto de **colecciones**. Cada colección aglutina a un conjunto de objetos (documentos) del mismo tipo, aunque como vimos en teoría, cada documento puede tener un conjunto de atributos diferente.

In [ ]:
posts = db.posts
posts

Importación de los ficheros CSV. Por ahora creamos una colección diferente para cada uno. Después estudiaremos cómo poder optimizar el acceso usando agregación.

In [ ]:
from urllib.request import urlretrieve

user = "dsevilla"
repo = "bd2-public"
src_dir = "util"
pyfile = "csv_to_mongo.py"
course="23-24"

url = f"https://raw.githubusercontent.com/{user}/{repo}/{course}/{src_dir}/{pyfile}"
urlretrieve(url, filename='csv_to_mongo.py')

In [ ]:
!cat csv_to_mongo.py

In [ ]:
import csv_to_mongo
help(csv_to_mongo)

In [ ]:
csv_to_mongo.csv_to_mongo('Posts.csv',db.posts)

In [ ]:
csv_to_mongo.csv_to_mongo('Users.csv',db.users)

In [ ]:
csv_to_mongo.csv_to_mongo('Votes.csv',db.votes)

In [ ]:
csv_to_mongo.csv_to_mongo('Comments.csv',db.comments)

In [ ]:
csv_to_mongo.csv_to_mongo('Tags.csv',db.tags)

In [ ]:
posts.count_documents({})

### El API de colección de MongoDB

El API de colección en Python se puede encontrar aquí: https://docs.mongodb.com/drivers/pymongo/. La mayoría de libros y referencias muestran el uso de mongo desde Javascript, ya que el *shell* de MongoDB acepta ese lenguaje. La sintaxis con respecto a Python cambia un poco, y se puede seguir en el enlace anterior. Existe incluso un curso de la MongoDB University que se puede realizar de forma gratuita: https://learn.mongodb.com/learning-paths/using-mongodb-with-python

## Framework de Agregación

Framework de agregación:
- Aquí está la referencia de las diferentes etapas por las que puede pasar un pipeline: https://www.mongodb.com/docs/manual/reference/operator/aggregation-pipeline/.
- Y aquí los distintos operadores que se permiten dentro de las etapas: https://docs.mongodb.com/manual/reference/operator/aggregation/.
- Y aquí incluso un libro completo con usos prácticos de ejecutar agregación: https://www.practical-mongodb-aggregations.com/.

A continuación otro vídeo interseante:

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('VSX4a3h4SmQ',width=600)

![](https://miro.medium.com/max/1060/1*2lDBxvZ8Cr3JYkoODTa0lQ.png)

Proyección:

In [ ]:
respuestas = db['posts'].aggregate( [
    {'$project' : { 'Id' : True }},
    {'$limit': 20} ])
list(respuestas)

_Lookup_!

In [ ]:
respuestas = posts.aggregate( [
        {'$match': { 'Score' : {'$gte': 40}}},
        {'$limit' : 50},
        {'$lookup': {
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"}
        }
        ])
list(respuestas)

El `$lookup` genera un _array_ con todos los resultados. El operador `$arrayElementAt` accede al primer elemento.

In [ ]:
respuestas = db.posts.aggregate( [
        {'$match': { 'Score' : {'$gte': 40}}},
        {'$limit' : 50},
        {'$lookup': {
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"}
        },
        { '$project' : {
            'Id' : True,
            'Score' : True,
            'username' : {'$arrayElemAt' : ['$owner.DisplayName', 0]},
            'owner.DisplayName' : True
          }}
        ])
list(respuestas)

`$unwind` también puede usarse. "Desdobla" cada fila por cada elemento del array. En este caso, como sabemos que el array sólo contiene un elemento, sólo habrá una fila por fila original, pero sin el _array_. Finalmente se puede proyectar el campo que se quiera.

In [ ]:
respuestas = db.posts.aggregate( [
        { '$match': { 'Score' : {'$gte': 40}}},
        {'$limit' : 50},
        { '$lookup': {
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"
          }
        },
        { '$unwind': '$owner'},
        { '$project' : {
             'username': '$owner.DisplayName'
          }
        }
        ])
list(respuestas)

## Definición de índices

Las consultas anteriores no se han podido realizar, ya que el uso de `$lookup` (equivalente a un JOIN de SQL) hace que se tenga que buscar en toda la colección usuarios para encontrar el usuario con `Id` especificado por `OwnerUserId`. Vamos a probarlas sin limitación después de crear un índice.


In [ ]:
db.users.create_index(['Id', pymongo.HASHED])

In [ ]:
respuestas = db.posts.aggregate( [
        { '$match': { 'Score' : {'$gte': 40}}},
        { '$lookup': {
            'from': "users",
            'localField': "OwnerUserId",
            'foreignField': "Id",
            'as': "owner"
          }
        },
        { '$unwind': '$owner'},
        { '$project' : {
             'username': '$owner.DisplayName'
          }
        }
        ])
list(respuestas)

Veamos cómo se ha usado el índice en la consulta (no incluyo la consulta entera porque el explain no lo termina de hacer bien si hay muchos pasos, no sé exactamente por qué).

In [ ]:
db.command('aggregate', 'posts', pipeline= [
  { '$match': { 'Score' : {'$gte': 40}}},
  { '$lookup': {
      'from': "users",
      'localField': "OwnerUserId",
      'foreignField': "Id",
      'as': "owner"
    }
  }],
  explain = True
)

Calculemos la media de puntuación por usuario:

In [ ]:
media_puntuación_usuarios = db.posts.aggregate([
    {
        '$group': {
            '_id': '$OwnerUserId',
            'media_puntuación': { '$avg':  '$Score' }
            }
    }])
list(media_puntuación_usuarios)

Calculemos el máximo y mínimo de puntuación de cada usuario:

In [ ]:
max_min_puntuación_usuarios = db.posts.aggregate([
    {
        '$group': {
            '_id': '$OwnerUserId',
            'media_puntuación': { '$avg':  '$Score' },
            'min': { '$min':  '$Score' },
            'max': { '$max' : "$Score"}
        }
    }])
list(max_min_puntuación_usuarios)

In [ ]:
!head Votes.csv

Podemos comprobar que todos los posts tienen los votos correctos contando los votos positivos (`VoteTypeId=2`) y negativos (`VoteTypeId=3`).

Primero, calcular el resultado de cada voto. Nótese cómo haciendo uso de `$cond` se puede aplicar una condición `if` para sumar o restar dependiendo del tipo del voto.

Después lo comprobaremos con los existentes en Posts.

In [ ]:
score_by_post = db.votes.aggregate([
    { "$match" : {"$or" : [ {"VoteTypeId" : 2},
                            {"VoteTypeId" : 3}]
                  }
    },
    {
        '$group': {
            '_id': '$PostId',
            'score': { '$sum': { "$cond": {
                                            "if" : { "$eq" : [ "$VoteTypeId", 2 ]},
                                            "then" : 1,
                                            "else" : -1
                                          }} }
        }
    },
    { '$limit' : 20}
    ])
list(score_by_post)

Y después comparado con lo que hay en los Posts. La consulta, como se puede ver, sacará sólo los elementos en donde los votos en `Votes` no coincide con `Posts` (debe ser vacío). Nótese el uso de la nueva construcción `$expr`.


 **CUIDADO** Si no se define el índice correcto, la consulta tarda mucho. ¿Cuál?

In [ ]:
score_by_post_comp = db.votes.aggregate([
    { "$match" : {"$or" : [ {"VoteTypeId" : 2},
                            {"VoteTypeId" : 3}]
                  }
    },
    {
        '$group': {
            '_id': '$PostId',
            'score': { '$sum': { "$cond" : {
                                            "if" : { "$eq" : [ "$VoteTypeId", 2 ]},
                                            "then" : 1,
                                            "else" : -1
                                           }}}
        }
    },
    {
        "$lookup": {
            'from': "posts",
            'localField': "_id",
            'foreignField': "Id",
            'as': "post"
        }
    },
    { "$unwind" : "$post"},
    {
        "$match" : {'$expr' : {'$ne' : [ '$score', '$post.Score']}}
    }
])
list(score_by_post_comp)

### Ejemplo de realización de una consulta de reciprocidad

Como ejemplo de consulta compleja con el Framework de Agregación, adjunto una posible solución a la consulta siguiente: ¿Hay usuarios que hacen preguntas y son respondidos por otros usuarios que a su vez preguntaron y fueron respondidos por los primeros? (Es decir, ¿hay reciprocidad en las interacciones en Stacokverflow español?)

In [ ]:
RQ4 = db.posts.aggregate( [
        { "$match" : {"PostTypeId": 2}},
        { '$lookup': {
            'from': "posts",
            'localField': "ParentId",
            'foreignField': "Id",
            'as': "question"
          }
        },

        { '$unwind' : '$question' },

        { '$project' : {
            'OwnerUserId': True,
            'OP' : '$question.OwnerUserId'
          }
        },
        { '$group' : {'_id' : {'min' : { '$min' : ['$OwnerUserId' , '$OP'] },
                               'max' : { '$max' : ['$OwnerUserId' , '$OP'] }},
                      'pairs' : {'$addToSet' : { '0q': '$OP', '1a': '$OwnerUserId'}}
          }
        },
        { '$project': {
            'pairs' : True,
            'npairs' : { '$size' : '$pairs'}
          }
        },
        { '$match' : { 'npairs' : { '$eq' : 2}} }
    ])
RQ4 = list(RQ4)
RQ4

En el caso de que queramos tener como referencia las preguntas y respuestas a las que se refiere la conversación, se puede añadir un campo más que guarde todas las preguntas junto con sus respuestas consideradas:

In [ ]:
RQ4 = db.posts.aggregate( [
        { '$match': { 'PostTypeId' : 2}},
        { '$lookup': {
            'from': "posts",
            'localField': "ParentId",
            'foreignField': "Id",
            'as': "question"
          }
        },

        { '$unwind' : '$question' },

        { '$project' : {
            'OwnerUserId': True,
            'QId' : '$question.Id',
            'AId' : '$Id',
            'OP' : '$question.OwnerUserId'
          }
        },
        { '$group' : {'_id' : {'min' : { '$min' : ['$OwnerUserId' , '$OP'] },
                               'max' : { '$max' : ['$OwnerUserId' , '$OP'] }},
                      'pairs' : {'$addToSet' : { '0q':'$OP', '1a': '$OwnerUserId'}},
                      'considered_pairs' : { '$push' : {'QId' : '$QId', 'AId' : '$AId'}}
          }
        },
        { '$project': {
            'pairs' : True,
            'npairs' : { '$size' : '$pairs'},
            'considered_pairs' : True
          }
        },
        { '$match' : { 'npairs' : { '$eq' : 2}} }
    ])
RQ4 = list(RQ4)
RQ4

In [ ]:
(db.posts.find_one({'Id': 238}), db.posts.find_one({'Id': 243}),
db.posts.find_one({'Id': 222}), db.posts.find_one({'Id': 223}))

### Ejemplo de consulta: Tiempo medio desde que se hace una pregunta hasta que se le da la primera respuesta

Veamos cómo calcular el tiempo medio desde que se hace una pregunta hasta que se le da la primera respuesta. No se considerarán pues las preguntas que no tienen respuesta, lo cual es razonable.

Esto sólo calcula el tiempo mínimo de cada pregunta a su respuesta. Después se añadirá el código que calcula la media final:

In [ ]:
min_answer_time = db.posts.aggregate([
    { "$match" : {"PostTypeId" : 2} },
    { '$group' : {
        '_id' : '$ParentId',
        # 'answers' : { '$push' : {'Id' : "$Id", 'CreationDate' : "$CreationDate"}},
        'min' : {'$min' : "$CreationDate"}
      }
    },
    { "$lookup" : {
        'from': "posts",
        'localField': "_id",
        'foreignField': "Id",
        'as': "post"}
    },
    { "$unwind" : "$post"},
    { "$project" : {
        "_id" : True,
        "min" : True,
        "diff" : {"$subtract" : ["$min", "$post.CreationDate"]}
      }
    }
])
min_answer_time = list(min_answer_time)
min_answer_time

La siguiente consulta sí calcula la media:

In [ ]:
avg_answer_time = db.posts.aggregate([
    { "$match" : {"PostTypeId" : 2} },
    { '$group' : {
        '_id' : '$ParentId',
        # 'answers' : { '$push' : {'Id' : "$Id", 'CreationDate' : "$CreationDate"}},
        'min' : {'$min' : "$CreationDate"}
      }
    },
    { "$lookup" : {
        'from': "posts",
        'localField': "_id",
        'foreignField': "Id",
        'as': "post"}
    },
    { "$unwind" : "$post"},
    { "$project" : {
        "_id" : True,
        "min" : True,
        "diff" : {"$subtract" : ["$min", "$post.CreationDate"]}
      }
    },
    #    { "$sort" : {'_id' : 1} }
    {
      "$group" :  {
        "_id" : None,
        "avg" : { "$avg" : "$diff"}
      }
    }
])
avg_answer_time = list(avg_answer_time)
avg_answer_time

## EJERCICIO 1: Listar aquellos usuarios con una reputación (`Reputation`) superior o igual a 500 y un número de visualizaciones (`Views`) superior a 200. De dichos usuarios solo mostrar los campos:
- `DisplayName`
- `Location`
- `Reputation`
- `Views`

*IMPORTANTE: No mostrar el campo `_id`*

## EJERCICIO 2: Listar todos post que son preguntas y sus votos asociados. Para ello, se permite definir los indices que se consideren adecuados.

*Ejemplo*: Suponiendo que el post con `_id` `65e6f1c8d93b356db73e5433` tuviera tres votos asociados, la solución debe devolver una salida de en este formato (nótese que los campos `CreationDate`, `Score` e `_id` se refieren al mismo post):

```json
{'CreationDate': datetime.datetime(2015, 10, 29, 15, 56, 52, 933000),
  'Score': 40,
  '_id': ObjectId('65e6f1c8d93b356db73e5433'),
  'post_votes': {'VoteTypeId': 2}},
 {'CreationDate': datetime.datetime(2015, 10, 29, 15, 56, 52, 933000),
  'Score': 40,
  '_id': ObjectId('65e6f1c8d93b356db73e5433'),
  'post_votes': {'VoteTypeId': 2}},
 {'CreationDate': datetime.datetime(2015, 10, 29, 15, 56, 52, 933000),
  'Score': 40,
  '_id': ObjectId('65e6f1c8d93b356db73e5433'),
  'post_votes': {'VoteTypeId': 16}}
```


## EJERCICIO 3: Listar los nombres de usuarios (campo `DisplayName` de `users`) que tienen más preguntas que respuestas

(Contando los usuarios debe salir exactamente 54483 usuarios).

## EJERCICIO 4: Comprobad, de forma similar a los ejemplos anteriores, que el campo `CommentCount` de `Posts` es correcto, comprobándolo con la colección `Comments`. Definid los índices necesarios.

El resultado de la consulta de los números de comentarios que difieren debe ser vacío como en el ejemplo de los votos anterior.
